In [ ]:
import pandas as pd
import numpy as np


In [ ]:
%%time
toxic_comments_df = pd.read_csv('train.csv')
#print initial data
tweets.head()

In [ ]:
toxic_comments_df[toxic_comments_df.target > .8].comment_text